<a href="https://colab.research.google.com/github/jihyunjeongme/deeplearning-tensflow-3min/blob/master/%5B3%EB%B6%84_%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_5%EC%9E%A5_%ED%85%90%EC%84%9C%EB%B3%B4%EB%93%9C%EC%99%80_%EB%AA%A8%EB%8D%B8_%EC%9E%AC%EC%82%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.1 학습 모델 저장하고 재사용하기

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
cd /content/gdrive/My Drive/Colab Notebooks/deeplearning-tensorflow-3mins

/content/gdrive/My Drive/Colab Notebooks/deeplearning-tensorflow-3mins


In [0]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


In [0]:
# loadtxt 함수를 이용하여 간단하게 데이터를 읽어들인 후 
# 1열과 2열은 x_data로, 3열부터 마지막 열까지는 y_data로 변환하였음.

In [0]:
# 신경망 모델을 구성할 차례
# 학습 횟수를 카운트 하는 변수(학습에 직접 사용되지는 않음)
# 변수 정의 시 trainable=False라는 옵션을 줌

global_step = tf.Variable(0, trainable=False, name='global_step')


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# 앞 장에서 보다 계층을 하나 더 늘리고, 편향은 없이 가중치만 사용한 모델로 만들어 봄
# 계층은 하나 늘었지만, 모델이 더 간략해졌으니 신경망의 구성이 조금 더 명확하게 드러남

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)\

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,
                                          logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

In [0]:
# 코드를 보면 두 번째 가중치인 W2의 형태가 [10, 20]인것을 볼 수있음.
# 이유는 앞단 계층의 출력 크기가 10이고, 뒷단 계층의 입력 크기가 20이기 때문
# global_step 매개변수에 앞서 정의한 global_step변수를 넘겨준 것을 확인
# 이렇게 하면 최적화 함수가 학습용 변수들을 최적화 할때마다
# global_step 변수의 값을 1씩 증가시키게 됨

In [0]:
# 모델 구성 끝
# 세션을 열고 최적화를 실행
# 모델을 불러들이고 저장하는 코드

########
# 신경망 모델 학습
#######

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [0]:
# tf.global_variables는 앞서 정의한 변수들을 가져오는 함수
# 이 함수를 써서 앞서 정의한 변수들을 모두 가져와서, 이후 변수들을 파일에 저장하거나
# 이전에 학습한 결과를 불러와 담는 변수들로 사용

In [0]:
# ./model 디렉터리에 기존에 학습해둔 모델이 있는지를 확인해서 
# 모델이 있다면 saver.restore 함수를 사용해 학습된 값들을 불러오고,
# 아니면 변수를 새로 초기화
# 학습된 모델을 저장한 파일을 체크포인트 파일 이라고 함

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  saver.restore(sess, ckpt.model_checkpoint_path)
else:
  sess.run(tf.global_variables_initializer())
  

In [0]:
# 최적화 수행
# 이전과는 달리 step 값이 아니라 global_step 값을 이용해 
# 학습을 몇 번째 진행하고 있는지를 출력
# global_step은 텐서 타입의 변수이므로 값을 가져 올 때 sess.run(global_step)을 이용

for step in range(2):
  sess.run(train_op, feed_dict={X: x_data, Y:y_data})
  
  print('Step: %d, ' % sess.run(global_step),
        'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
  

Step: 1,  Cost: 1.220
Step: 2,  Cost: 1.151


In [0]:
# 마지막으로 최적화가 끝난 뒤 학습된 변수들을 지정한 체크포인트 파일에 저장
# 두 번쨰 매개변수는 체크포인트 파일의 위치와 이름
# global_step의 값은 저장할 파일의 이름을 추가로 붙게 되며
# 텐서 변수 또는 숫자값을 넣어줄 수 있습니다.
# 이를 이용해 여러 상태의 체크포인트를 만들 수 있고, 가장 효과적인 체크포인트를 선별,사용


saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-2'

In [0]:
# 예측 결과와 정확도 확인

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data,
                                                       Y: y_data}))

예측값: [0 1 1 0 0 1]
실제값: [0 1 2 0 0 2]
정확도: 66.67


In [0]:
# 총 10번을 실했해을 때 정확도가 100.00으로 나옴
# gobal_step으로 저장한 값을 불러와서 증가시키면 step이 늘어남
# 텐서플로를 이용하면 이렇게 쉽게 학습시킨 모델을 저장하고 불러와서
# 재사용 할수 있음.
# 이 방식을 이용해 모델 구성, 학습, 예측 부분을 각각 분리하여 학습을 따로 한 뒤
# 예측만 단독으로 실행하는 프로그램을 작성할 수 있음

# 5.2 텐서보드 사용하기